# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 6: Building an interactive dashboard using Plotly and Dash


## Objectives


In this lab, we will build a dashboard using pandas dash package and plot charts using Plotly..

*   Build an interactive dashboard app using pandas dash.
*   Plot interactive charts in the dashboard using Plotly.


***

Install the necessary packages

In [ ]:
!pip install pandas dash
!pip install wget

Download the dataset

In [ ]:
!wget -O spacex_launch_dash.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv

--2021-08-20 20:12:49--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2021-08-20 20:12:51 (480 MB/s) - ‘spacex_launch_dash.csv’ saved [3033/3033]



Inspect the dataset

In [ ]:
import pandas as pd
spacex_data = pd.read_csv('/content/spacex_launch_dash.csv')
spacex_data.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


Install ngrok for running the app in a separate tab in your browser

In [ ]:
### Install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

### Run ngrok to tunnel Dash app port 8050 to the outside world. 
### This command runs in the background.
get_ipython().system_raw('./ngrok http 8050 &')

### Get the public URL where you can access the Dash app. Copy this URL.
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-08-20 20:50:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.45.159.115, 34.226.109.249, 54.243.172.172, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.45.159.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.19M  6.55MB/s    in 2.0s    

2021-08-20 20:50:43 (6.55 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://a255553ec284.ngrok.io


Build the dash app and save it as `spacex_dash_app.py`

In [ ]:
%%writefile spacex_dash_app.py
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                html.Label('Dropdown'),
                                dcc.Dropdown(id='site-dropdown',
                                    options=[
                                             {'label':'All Sites', 'value':'All Sites'},
                                             {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                             {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                             {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                             {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'}
                                    ],
                                    value='All Sites',
                                    placeholder='Select a Launch Site here',
                                    searchable=True
                                ),
                                html.Br(),

                                # Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000,
                                                marks={i: '{} (Kg)'.format(i) for i in range(0, 10001, 1000)},
                                                value=[min_payload, max_payload]),

                                # Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))
def get_pie(site_sel):
  if site_sel == 'All Sites':
    piedata = spacex_df[['Launch Site', 'class']]
    piedat = piedata.groupby(['Launch Site']).sum()
    pie_fig = px.pie(piedat, values="class", names=piedat.index, title='Total successful launches by site')
    return pie_fig
  else:
    piedata = spacex_df[spacex_df['Launch Site']==site_sel]
    piedat = piedata['class'].value_counts()
    pie_fig = px.pie(piedat, names=piedat.index, values=piedat.values, title='Total successful launches for site {}'.format(site_sel))
    return pie_fig

# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
               [Input(component_id='site-dropdown', component_property='value'),
                Input(component_id='payload-slider', component_property='value')])
def get_sct(sel_site, payload):
  if sel_site == 'All Sites':
    sctdata = spacex_df[['Launch Site', 'class', 'Payload Mass (kg)', 'Booster Version Category']]
    sctdat = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload[0]) & (spacex_df['Payload Mass (kg)'] <= payload[1])]
    sct_fig = px.scatter(sctdat, x="Payload Mass (kg)", y="class", color="Booster Version Category")
    return sct_fig
  else:
    sctdata = spacex_df[['Launch Site', 'class', 'Payload Mass (kg)', 'Booster Version Category']]
    sctdat1 = sctdata[sctdata['Launch Site']==sel_site]
    sctdat = sctdat1[(sctdat1['Payload Mass (kg)'] >= payload[0]) & (sctdat1['Payload Mass (kg)'] <= payload[1])]
    sct_fig = px.scatter(sctdat, x="Payload Mass (kg)", y="class", color="Booster Version Category")
    return sct_fig
# Run the app
if __name__ == '__main__':
    app.run_server()

Writing spacex_dash_app.py


Start running the app, then click on the link obtained from ngrok

In [ ]:
!python spacex_dash_app.py

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "spacex_dash_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Aug/2021 20:51:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2021 20:51:06] "GET /_dash-component-suites/dash/deps/react@16.v1_21_0m1629469491.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2021 20:51:06] "GET /_dash-component-suites/dash/deps/polyfill@7.v1_21_0m1629469491.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2021 20:51:07] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_17_1m1629469491.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2021 20:51:07] "GET /_dash-component-suites/dash/deps/prop-types@15.v1_21_0m1629469491.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2021 20:51:07] "GET /_dash-component-suites/dash/deps/react-dom@16.v1_21_0m1629469491.14.0.min.js HTTP/1.1" 200 -
127.0